In [2]:
import requests
from bs4 import BeautifulSoup as bs
import time
from pymongo import MongoClient
import lxml

headers = {'accept': '*/*',
           'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko)\
            Chrome/51.0.2704.103 Safari/537.36'
          }

list_id = []


def add_mongo(list_data, list_id):
    '''
    добавляет элементы в базу данных
    :param list_data: список элементов которые нужно вставить
    :param list_id: список с вставленными элементами
    :return:
    '''
    client = MongoClient('mongodb://127.0.0.1:27017')
    db = client['My_Study']
    docs = db.study
    id_add = list_data[0]
    brief = list_data[1]
    prices = int(list_data[2].replace(' ',''))
    description = list_data[3]
    url_add = list_data[4]    
    for i in docs.find():
        list_id.append(i['id_number'])
        
    if id_add not in list_id:
        list_id.append(id_add)
        doc_data = {
            'id_number': id_add,
            'brief': brief,
            'price': prices,
            'description': description,
            'url_add': url_add
        }
        docs.insert_one(doc_data)
    return list_id




In [3]:
def page_num():
    ''' Расчитывает количество страниц с обявлениями
    заданных параметров'''
    url = f"https://www.avito.ru/moskva/doma_dachi_kottedzhi/prodam?cd=1&pmax=2500000&pmin=0&\
            f=556_5735-5736.584_14553b14556.1246_14591b14598.1248_0b14655"
    
    try:
        reg = requests.get(url)
        soup = bs(reg.text, 'lxml')
        a = []
        s1 = soup.find_all('div', {'class': 'pagination-pages clearfix'})
        for i in soup.findAll('a', {'class': 'pagination-page'}):
            a.append(i.text)
        return len(a)
    
    except requests.exceptions.ConnectionError:
        print('Please check your internet connection!')
        exit(1)








In [5]:
def data_publication(url_publication):
    '''Получает внутреннюю ссылку на объявление
    добавляет к внешенй ссылке avito и заходит на объявления
    после чего получает ключевые данные из объявления
    '''
    list_data = []
    url = f'https://www.avito.ru{url_publication}'
    data_ = requests.get(url)
    try:
        soup = bs(data_.text, 'lxml')    
        if soup.find('div', {'class': 'item-view-search-info-redesign'}): 
            list_data.append(soup.find('div', {'class': 'item-view-search-info-redesign'}).span.text)
        else:
            list_data.append('')
            
        if soup.find('span', {'class': 'title-info-title-text'}):
            list_data.append(soup.find('span', {'class': 'title-info-title-text'}).text)
        else:
            list_data.append('')
            
        if soup.find('span', {'class': 'js-item-price'}):
            list_data.append(soup.find('span', {'class': 'js-item-price'}).text)
        else:
            list_data.append('')
            
        if soup.find('div', {'class': 'item-description-text'}):
            list_data.append(soup.find('div', {'class': 'item-description-text'}).text)
        else:
            list_data.append('')
            
        list_data.append(url)
        time.sleep(1)
        return list_data
    
    except requests.exceptions.ConnectionError:
        print('Please check your internet connection!')
        exit(1)



In [6]:
# Ищем обьявления и добавляем в базу
def publication():
    for num_page in range(1, page_num()):
        url = f'https://www.avito.ru/moskva/doma_dachi_kottedzhi/prodam?&p={num_page}cd=1&pmax=2500000&pmin=0&f=556_5735-5736.584_14553b14556.1246_14591b14598.1248_0b14655'
        req = requests.get(url, headers)
        soup = bs(req.text, 'xml')
        
        for i in soup.findAll('div', {'class': 'catalog-content '}):
            
            for x in i.find_all('h3'):
                url_publication = x.find("a")["href"]  # ищем текст ссылки и добавляем
                list_data = data_publication(url_publication)  # функция возвраащет список параметров
                add_mongo(list_data, list_id)  # функция добавляет в базу данные
                
publication()

In [7]:
# Ищем и выводим  дома по цене
price_ = int(input('Введите цену: '))


def price_search(price_):
    client = MongoClient('mongodb://127.0.0.1:27017')
    db = client['My_Study']
    docs = db.study
    
    for i in docs.find():
        try:
            if i['price'] <  price_:
                print(i)
                
        except KeyError:
            print(i)
            
price_search(price_)

Введите цену: 0
